In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector, array_to_latex, plot_histogram
import numpy as np

%matplotlib inline

simulator = Aer.get_backend('qasm_simulator')

state_vector = Statevector.from_label('0011')
circuit.measure_all()
result = execute(circuit, backend=simulator, shots=10000).result()
display(plot_bloch_multivector(state_vector))
display(plot_histogram(result.get_counts()))